# 🎅 Welcome to the North Pole Data Governance Office!
## Unity Catalog Functions: Secure, Reusable Tools for Santa's Workshop

---

### 📬 The Challenge
Santa receives **millions of letters** from children worldwide, containing sensitive information:
* 👶 **Names** - Personal identifiers that need protection
* 🏠 **Locations** - Cities and provinces where children live

### ✨ The Solution: Unity Catalog Functions
The North Pole Modernization Office (NPMO) uses **Unity Catalog Functions** to create governed, reusable data tools that:
* ✅ **Protect PII** - Automatic masking of sensitive data
* ✅ **Enable Safe Queries** - Governed access patterns
* ✅ **Track Lineage** - Unity Catalog monitors all usage
* ✅ **Reusable Everywhere** - SQL, Python, dashboards, and applications

### 🎯 What You'll Learn
* How to create **scalar functions** for data masking
* How to build **aggregate functions** for governed queries
* How to create **table-valued functions** for safe search

---

## 🏗️ Architecture Overview

```
┌──────────────┐     ┌─────────────────────────────────────────────────────────────────────────┐     ┌──────────────┐
│ 📬 Letters   │     │                    Data Intelligence Platform                           │     │ 🛡️ Masked    │
│ 👶 Names     │ ──► │  [1.Scalar]──►[2.Aggregate]──►[3.Table-Valued]──►[4.Governed Views]    │ ──► │ 📊 Analytics │
│ 🏠 Locations │     │     mask_name()   get_province()   search_letters()   letters_masked   │     │ 🔍 Search    │
└──────────────┘     │  ─────────────────────────────────────────────────────────────────────  │     └──────────────┘
                     │  [Delta Lake] [Apache Spark™] [Unity Catalog] [Row/Column Security]    │
                     └─────────────────────────────────────────────────────────────────────────┘
```

📎 **Full architecture diagram:** [uc_functions_architecture.html](uc_functions_architecture.html)

---

*"Security isn't just policy at the North Pole - it's built into the data platform!"* 🎄✨

## 🎁 Step 0: Configuration Setup

### What's Happening Here?
Before we build our governed functions, we need to set up our workspace.

**We're configuring:**
* 📚 **Catalog & Schema**: Where our functions will live
* 📋 **Source Table**: Letters from Canadian children

🎯 **Pro Tip**: Update the configuration below to match your environment!

In [0]:
# 🎄 CONFIGURATION - Update these values for your environment!
TARGET_CATALOG = "main"
TARGET_SCHEMA = "dbrx_12daysofdemos"
TARGET_VOLUME = "raw_data_volume"
SOURCE_TABLE_NAME = "santa_letters_canada"

# Derived names (no need to change)
full_schema = f"{TARGET_CATALOG}.{TARGET_SCHEMA}"
source_table = f"{full_schema}.{SOURCE_TABLE_NAME}"

print("🎅 North Pole UC Functions Configuration")
print("=" * 60)
print(f"📚 Catalog: {TARGET_CATALOG}")
print(f"📁 Schema: {TARGET_SCHEMA}")
print(f"📋 Source Table: {source_table}")
print("=" * 60)

In [0]:
%run "../00-init/load-data"

In [0]:
# Set up the catalog and schema
spark.sql(f"USE CATALOG {TARGET_CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {full_schema}")
spark.sql(f"USE SCHEMA {TARGET_SCHEMA}")

print(f"✅ Using catalog: {TARGET_CATALOG}")
print(f"✅ Using schema: {full_schema}")

## 📊 Step 1: Explore the Raw Data (The Problem!)

### What's Happening Here?
Let's look at our raw letter data to understand **why we need governance**.

⚠️ **The Problem**: This raw data is NOT safe to share with:
* External dashboards
* Demo environments
* Third-party applications

In [0]:
# ⚠️ RAW DATA - Contains visible PII!
print("⚠️ Raw data with visible PII - NOT safe to share!\n")

df = spark.sql(f"""
SELECT
  name,
  city,
  province,
  LEFT(letter, 150) AS letter_preview,
  gifts
FROM {source_table}
LIMIT 10
""")
display(df)

---

# 🔒 Part 1: Building PII Protection Functions

---

## What Are UC Scalar Functions?

**Scalar functions** take input values and return a single output value. Perfect for:
* Data masking (hiding sensitive parts)
* Data transformation (formatting, cleaning)
* Calculations (derived values)

**The Magic:** Once created, these functions can be used everywhere - SQL, Python, dashboards! 🎅

## 👶 Function 1: `mask_name()` - Name Anonymization

### Why This Matters
Children's names require special protection:
* 🛡️ **Child Privacy Laws** - Extra protection for minors
* 🎭 **Anonymization** - Enable analytics without identifying individuals

### How It Works
* **Input:** `Emma`
* **Output:** `E**a`
* Shows first and last character only

In [0]:
# 🎭 Create the mask_name() function
spark.sql("""
CREATE OR REPLACE FUNCTION mask_name(name STRING)
RETURNS STRING
COMMENT 'Masks personal names for privacy - shows first and last character only'
RETURN
  CASE
    WHEN name IS NULL OR TRIM(name) = '' THEN NULL
    WHEN LENGTH(TRIM(name)) <= 2 THEN REPEAT('*', LENGTH(TRIM(name)))
    ELSE CONCAT(
      SUBSTRING(TRIM(name), 1, 1),
      REPEAT('*', LENGTH(TRIM(name)) - 2),
      SUBSTRING(TRIM(name), -1)
    )
  END
""")

print("✅ Created function: mask_name()")

In [0]:
# 🧪 Test mask_name()
print("🧪 Testing mask_name() function:\n")

df = spark.sql("""
SELECT
  'Emma' AS original_name,
  mask_name('Emma') AS masked_name
UNION ALL
SELECT
  'Alexander' AS original_name,
  mask_name('Alexander') AS masked_name
UNION ALL
SELECT
  'Jo' AS original_name,
  mask_name('Jo') AS masked_name
""")
display(df)

### 💡 Why UC Functions for Masking?

| Traditional Approach | UC Functions |
|---------------------|---------------|
| Masking logic in application code | Masking built into the data layer |
| Different implementations per system | Same function everywhere |
| No audit trail | Full Unity Catalog lineage |
| Easy to forget/bypass | Automatic when function is used |

---

# 📊 Part 2: Aggregate Functions for Governed Queries

---

## Why Aggregate Functions?

Users often need to answer questions like:
* *"How many letters from Ontario?"*
* *"What's the most popular gift in Quebec?"*

**The Solution:** Create functions that return **governed, pre-aggregated data**.

## 🗺️ Function 2: `get_province_summary()` - Governed Stats

### How It Works
* **Input:** Province name (e.g., 'Ontario')
* **Output:** JSON with letter count, cities, sample gifts

### Why JSON Output?
* 📊 **Structured** - Consistent format every time
* 🔒 **Controlled** - Only returns approved fields

In [0]:
# 📊 Create the get_province_summary() function
spark.sql(f"""
CREATE OR REPLACE FUNCTION get_province_summary(province_name STRING)
RETURNS STRING
COMMENT 'Returns JSON summary of letters from a specific province'
RETURN (
  SELECT TO_JSON(
    STRUCT(
      province_name AS province,
      COUNT(*) AS total_letters,
      COUNT(DISTINCT city) AS unique_cities,
      SLICE(COLLECT_SET(SUBSTRING(gifts, 1, 50)), 1, 10) AS sample_gifts
    )
  )
  FROM {source_table}
  WHERE UPPER(province) = UPPER(province_name)
     OR province = province_name
)
""")

print("✅ Created function: get_province_summary()")

In [0]:
# 🧪 Test get_province_summary()
print("🧪 Testing get_province_summary() function:\n")

df = spark.sql("SELECT get_province_summary('Ontario') AS ontario_stats")
display(df)

In [0]:
# 🗺️ Compare multiple provinces
print("🗺️ Province Comparison:\n")

df = spark.sql("""
SELECT 'Ontario' AS province, get_province_summary('Ontario') AS stats
UNION ALL
SELECT 'Quebec' AS province, get_province_summary('Quebec') AS stats
UNION ALL
SELECT 'British Columbia' AS province, get_province_summary('British Columbia') AS stats
""")
display(df)

---

# 🔍 Part 3: Table-Valued Functions for Safe Search

---

## What Are Table-Valued Functions?

Unlike scalar functions (which return one value), **table-valued functions** return **entire result sets**.

**Perfect for:**
* Search functionality (find matching records)
* Filtered views (subset of data)
* Complex queries with automatic masking

## 🔎 Function 3: `search_letters()` - Safe Keyword Search

### How It Works
* **Input:** Keyword to search (e.g., 'bicycle')
* **Output:** Results with AUTOMATICALLY MASKED names!

### Built-In Safety
* 🎭 Names are masked using our `mask_name()` function
* 📝 Letter previews have names replaced with masked versions
* 🔢 Limited to 10 results per search

In [0]:
# 🔍 Create the search_letters() table-valued function
spark.sql(f"""
CREATE OR REPLACE FUNCTION search_letters(keyword STRING)
RETURNS TABLE(
  masked_name STRING,
  city STRING,
  province STRING,
  letter_preview STRING
)
COMMENT 'Searches letters by keyword and returns results with masked names'
RETURN
  SELECT
    mask_name(name) AS masked_name,
    city,
    province,
    SUBSTRING(REGEXP_REPLACE(letter, name, mask_name(name)), 1, 200) AS letter_preview
  FROM {source_table}
  WHERE UPPER(letter) LIKE CONCAT('%', UPPER(keyword), '%')
     OR UPPER(gifts) LIKE CONCAT('%', UPPER(keyword), '%')
  LIMIT 10
""")

print("✅ Created function: search_letters()")

In [0]:
# 🧪 Test search_letters() - Bicycles
print("🔍 Searching for: 'bicycle'\n")

df = spark.sql("SELECT * FROM search_letters('bicycle')")
display(df)

In [0]:
# 🧪 Test search_letters() - LEGO
print("🔍 Searching for: 'LEGO'\n")

df = spark.sql("SELECT * FROM search_letters('LEGO')")
display(df)

In [0]:
# 🧪 Test search_letters() - Nintendo
print("🔍 Searching for: 'Nintendo'\n")

df = spark.sql("SELECT * FROM search_letters('Nintendo')")
display(df)

### 💡 Notice the Magic!

**All results have automatically masked names!**

* ✅ Never sees real child names
* ✅ Gets useful, relevant results
* ✅ All usage is tracked in Unity Catalog

---

# 🎯 Part 4: Putting It All Together

---

## Verify All Functions

In [0]:
# 📋 List all functions we created
print("📋 UC Functions in our schema:\n")

df = spark.sql("SHOW USER FUNCTIONS")
display(df)

## 🔒 Create a Governed View

Combine our masking functions into a **reusable view** that's always safe to query!

In [0]:
# 🔒 Create a masked view using our functions
masked_view = f"{full_schema}.holiday_letters_masked"

spark.sql(f"""
CREATE OR REPLACE VIEW {masked_view} AS
SELECT
  mask_name(name) AS child_name,
  city,
  province,
  REGEXP_REPLACE(letter, name, mask_name(name)) AS letter,
  gifts
FROM {source_table}
""")

print(f"✅ Created governed view: {masked_view}")

In [0]:
# 👀 Query the masked view
print("🔒 Data from governed view (all PII masked):\n")

df = spark.sql(f"SELECT * FROM {masked_view} LIMIT 10")
display(df)

## 📊 Analytics with Governed Data

Run analytics on the masked view - safe to share anywhere!

In [0]:
# 📊 Letters by province (safe to share!)
print("📊 Letters by Province:\n")

df = spark.sql(f"""
SELECT
  province,
  COUNT(*) AS letter_count,
  COUNT(DISTINCT city) AS unique_cities
FROM {masked_view}
GROUP BY province
ORDER BY letter_count DESC
""")
display(df)

In [0]:
# 🎁 Top gift requests (anonymized)
print("🎁 Top Gift Requests (with anonymized requesters):\n")

df = spark.sql(f"""
SELECT
  gifts,
  COUNT(*) AS request_count,
  SLICE(COLLECT_LIST(child_name), 1, 5) AS sample_requesters
FROM {masked_view}
WHERE gifts IS NOT NULL
GROUP BY gifts
ORDER BY request_count DESC
LIMIT 15
""")
display(df)

# 🎊 Congratulations! You've Built a Governed Data Platform!

---

## 🎓 What You Learned

### ✅ Functions Created
| Function | Type | Purpose |
|----------|------|--------|
| `mask_name()` | Scalar | Anonymize personal names |
| `get_province_summary()` | Scalar (JSON) | Governed aggregate stats |
| `search_letters()` | Table-valued | Safe search with auto-masking |

---

## 🔑 Key Takeaways

| Traditional | UC Functions |
|-------------|---------------|
| Masking in app code | Masking built into data layer |
| Different per system | Same function everywhere |
| No lineage tracking | Full Unity Catalog lineage |
| Easy to bypass | Enforced at query time |

---

## 🎅 From the North Pole Team

*"Santa checks his list twice, but with Unity Catalog Functions, the data checks itself! Now that's what we call governance magic."*

**- Mrs. Claus, Chief Data Officer**

---

### 🎄 Happy Holidays from Databricks! 🎄